# Attention Mechanisme Text to Text Translation : English to French

This notebook trains a model for French to English translation. Using the attention mechanism, the model learns to align and translate French text to English text.

---

## Import Required Libraries

We will start by importing the libraries we need for this project. You can install any missing libraries using the requirements.txt file provided or by running ``make install`` in the terminal.

In [6]:
%load_ext autoreload
%aimport utils.text_processing
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from datasets import load_dataset
from tqdm import tqdm

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from string import digits
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from utils.text_processing import TextProcessor

from tensorflow.keras.preprocessing.text import Tokenizer

import time
import re
import os

import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

%matplotlib inline
pd.set_option('display.max_colwidth', 200)

/home/idriss/Documents/Github/Artorius/to-delete/TALN/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-18 16:45:55.414261: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-18 16:45:55.415383: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-18 16:45:55.439152: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-18 16:45:55.439724: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU in

### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a cloud environment with GPU support. Run the next cell, and verify that the device_type is "GPU".

In [2]:
import tensorflow as tf
print("cuda available: ", tf.config.list_physical_devices('GPU'))

cuda available:  []


2025-05-18 16:45:56.856345: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-18 16:45:56.856647: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


### Verify TensorFlow CUDA and cuDNN Versions
Run the next cell to inspect which CUDA and cuDNN versions TensorFlow was built against:

In [3]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
build_info = tf.sysconfig.get_build_info()
print("CUDA version built:", build_info.get("cuda_version", "Unknown"))
print("cuDNN version built:", build_info.get("cudnn_version", "Unknown"))
print("Detected GPU devices:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.13.0
CUDA version built: 11.8
cuDNN version built: 8
Detected GPU devices: []


We provide a in depth analysis of the data in the ``exploratory_analysis.ipynb`` notebook. We will not be doing any exploratory analysis in this notebook. Instead, we will focus on building our baseline model. So, let's start by importing the dataset we will be using.

In [ ]:
dataset = load_dataset("Nicolas-BZRD/Parallel_Global_Voices_English_French", split='train').to_pandas()
dataset.head(10)

The actual data contains over 350,000 sentence-pairs. However, to speed up training for this notebook, we will only use a small portion of the data. 

In [ ]:
# TODO : Use the whole dataset (but it's too big for my computer)
dataset = dataset.sample(n=50000, random_state=42)
print(dataset.shape)

## Text Pre-Processing

The text pre-processing steps will be implemented in a class called ``TextPreprocessor``. This class will be used to clean and tokenize the text data. The class will also be used to convert the text to sequences and pad the sequences to a maximum length. This way we will be able to improve our model's without having to copy and paste the same code over and over again.

In [ ]:
max_sequence_length = 20

In [ ]:
dataset['en'] = TextProcessor(dataset, 'en').transform()
dataset['fr'] = TextProcessor(dataset, 'fr').transform()

dataset.head(10)

In [ ]:
# keep only sentences with less than max_sequence_length words
dataset = dataset[dataset['en'].str.split().str.len() <= max_sequence_length]
dataset = dataset[dataset['fr'].str.split().str.len() <= max_sequence_length]

In [ ]:
dataset

### Text to Sequence Conversion

To feed our data to a Seq2Seq model, we will have to convert both the input and the output sentences into integer sequences of fixed length. Check the exploratory data analysis notebook to see the distribution of the lengths of the sentences in the dataset. Based on that, we decided to fix the maximum length of each sentence to 20 since the average length of the sentences in the dataset is around 20.

We will use the ``Tokenizer`` class from the ``tensorflow.keras.preprocessing.text`` module to tokenize the text data. The ``Tokenizer`` class will also be used to convert the text to sequences. We will use the ``pad_sequences`` function from the same module to pad the sequences to the maximum length.

In [ ]:
def tokenization(lines, max_vocab_size=100000):
    tokenizer = Tokenizer(filters='', num_words=max_vocab_size)
    tokenizer.fit_on_texts(lines)
    return tokenizer

def encode_sequences(tokenizer, length, lines):
    seq = tokenizer.texts_to_sequences(lines)
    seq = pad_sequences(seq, maxlen=length, padding='post', truncating='post')
    return seq

def decode_sequences(tokenizer, sequence):
    text = tokenizer.sequences_to_texts([sequence])[0]
    return text

def get_most_common_words(tokenizer, n=10):
    word_counts = sorted(tokenizer.word_counts.items(), key=lambda x: x[1], reverse=True)
    return word_counts[:n]

In [ ]:
max_vocab_size = 5000

In [ ]:
dataset['en'] = dataset['en'].apply(lambda s: '<start> ' + s + ' <end>')
dataset['fr'] = dataset['fr'].apply(lambda s: '<start> ' + s + ' <end>')

In [ ]:
# Tokenize the English sentences
eng_tokenizer = tokenization(dataset["en"], max_vocab_size=max_vocab_size)
eng_vocab_size = len(eng_tokenizer.word_index) + 1

# Tokenize the French sentences
fr_tokenizer = tokenization(dataset["fr"], max_vocab_size=max_vocab_size)
fr_vocab_size = len(fr_tokenizer.word_index) + 1

In [ ]:
print('English Vocabulary Size: %d' % eng_vocab_size)
print('French Vocabulary Size: %d' % fr_vocab_size)

In [ ]:
print("Most common words in English: ", get_most_common_words(eng_tokenizer))
print("Most common words in French: ", get_most_common_words(fr_tokenizer))

## Model Building

We will now split the data into train and test set for model training and evaluation, respectively. We will use the ``train_test_split`` function from the ``sklearn.model_selection`` module to split the data. We will use 10% of the data for testing and the rest for training. We will also set the ``random_state`` parameter to 42 to ensure reproducibility. 

In [ ]:
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

It's time to encode the sentences. We will encode French sentences as the input sequences and English sentences as the target sequences. It will be done for both tra and test datasets.

In [ ]:
# prepare training data
trainX = encode_sequences(fr_tokenizer, max_sequence_length, dataset["fr"])
trainY = encode_sequences(eng_tokenizer, max_sequence_length, dataset["en"])

# prepare validation data
testX = encode_sequences(fr_tokenizer, max_sequence_length, test_data["fr"])
testY = encode_sequences(eng_tokenizer, max_sequence_length, test_data["en"])

In [ ]:
trainX = trainX.reshape((-1, trainX.shape[1], 1))
testX = testX.reshape((-1, testX.shape[1], 1))

trainY = trainY.reshape((-1, trainY.shape[1], 1))
testY = testY.reshape((-1, testY.shape[1], 1))

In [ ]:
testX.shape

In [ ]:
trainX.shape

In [ ]:
test_data

In [ ]:
def find_sentences_without_start_end(df, column):
    # Returns rows where the sentence does not start with <start> or does not end with <end>
    mask = ~df[column].str.startswith('<start>') | ~df[column].str.endswith('<end>')
    return df[mask]

# Example usage for English and French columns:
invalid_en = find_sentences_without_start_end(train_data, 'en')
invalid_fr = find_sentences_without_start_end(train_data, 'fr')
print("English sentences without <start>/<end>:", len(invalid_en))
print("French sentences without <start>/<end>:", len(invalid_fr))


In [ ]:
# decode sample sequences from the training set
for i in range(1500):
    english = decode_sequences(eng_tokenizer, trainY[i, : ,0])
    french = decode_sequences(fr_tokenizer, trainX[i, : ,0])
    print('English: ', english, len(english.split()))
    print('French: ', french , len(french.split()))
    print('---')

In [ ]:
print('<start>' in fr_tokenizer.word_index)
print(fr_tokenizer.word_index.get('<start>'))


## Encoder Decoder with Attention mechanism 

![Attention](../images/attention.jpg)

In [ ]:
class Encoder(tf.keras.Model):
    
    def __init__(self, vocab_size, embedding_dim, encoder_units, batch_size):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.encoder_units = encoder_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(encoder_units, 
                                      return_sequences=True,
                                      return_state=True,                                      
                                      recurrent_initializer='glorot_uniform')
    
    def call(self, x, hidden):
        # pass the input x to the embedding layer
        x = self.embedding(x)
        # pass the embedding and the hidden state to GRU
        output, state = self.gru(x, initial_state=hidden)
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_size, self.encoder_units))

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
    
    def __init__(self, units):
        
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        # query hidden state shape == (batch_size, hidden size)
        # query_with_time_axis shape == (batch_size, 1, hidden size)
        # values shape == (batch_size, max_len, hidden size)
        # we are doing this to broadcast addition along the time axis to calculate the score
        query_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(self.W1(query_with_time_axis) + self.W2(values)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [ ]:
class Decoder(tf.keras.Model):
    
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

        # used for attention
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
        
        # hidden state shape == (batch_size, hidden size)
        # enc_output shape == (batch_size, max_length, hidden_size)
        context_vector, attention_weights = self.attention(hidden, enc_output)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state, attention_weights

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [ ]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0
    with tf.GradientTape() as tape:
        # enc_output (batch_size, max?_lenght, encoder_units) ,enc_hidden (batch_size, encoder_units)
        enc_output, enc_hidden = encoder(inp, enc_hidden) 
        dec_hidden = enc_hidden

        # dec_input(batch_size, 1)
        dec_input = tf.expand_dims([fr_tokenizer.word_index['<start>']] * BATCH_SIZE, 1) 

        # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]):
            # passing enc_output to the decoder
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

            loss += loss_function(targ[:, t], predictions)
            dec_input = tf.expand_dims(targ[:, t], 1) # using teacher forcing

    batch_loss = (loss / int(targ.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

In [ ]:
BATCH_SIZE = 64
embedding_dim = 256
units = 1024

# Create data in memeory and shuffles the data in the batches
dataset=tf.data.Dataset.from_tensor_slices((trainX.reshape((-1, trainX.shape[-2])), trainY.reshape((-1, trainY.shape[1])))).shuffle(BATCH_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
encoder = Encoder(fr_vocab_size, embedding_dim=embedding_dim, encoder_units=units, batch_size=BATCH_SIZE)
decoder = Decoder(vocab_size=eng_vocab_size, embedding_dim=embedding_dim, dec_units=units, batch_sz=BATCH_SIZE)

In [ ]:
checkpoint_dir = '../models/encode_decoder_attention_training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [ ]:
EPOCHS = 10
steps_per_epoch = len(trainX) // BATCH_SIZE

for epoch in range(EPOCHS):
    start = time.time()

    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0
    for (batch, (inp, targ)) in tqdm(enumerate(dataset.take(steps_per_epoch)), total=steps_per_epoch):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1, total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

### Make Prediction using the trained Model

In [ ]:
def predict(sentences, source_sentence_tokenizer=fr_tokenizer, target_sentence_tokenizer=eng_tokenizer): 
    max_length = trainX.shape[1]
    attention_plot = np.zeros((max_length, max_length))

    sentence = TextProcessor(sentences, 'fr').process(sentences)

    inputs = [source_sentence_tokenizer.word_index[w] for w in sentences.split()]
    inputs = pad_sequences([inputs], maxlen=max_length, padding="post")
    inputs = tf.convert_to_tensor(inputs)

    result = ""
    
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([target_sentence_tokenizer.word_index['<start>']], 0)
    
    for t in range(max_length):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)

        # storing the attention weights to plot later on
        attention_weights =  tf.reshape(attention_weights,(-1, ))
        attention_plot [t] = attention_weights.numpy()
        
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += target_sentence_tokenizer.index_word[predicted_id] + ' '

        if target_sentence_tokenizer.index_word[predicted_id] == '<end>':
            return result, sentence, attention_plot

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

In [ ]:
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap="viridis")
    
    sentence = sentence.split()
    predicted_sentence = predicted_sentence.split()
    
    # Add <PAD> token if sentence or predicted_sentence is shorter than attention matrix
    if len(sentence) < attention.shape[1]:
        sentence += ["<PAD>"] * (attention.shape[1] - len(sentence))
        
    if len(predicted_sentence) < attention.shape[0]:
        predicted_sentence += ["<PAD>"] * (attention.shape[0] - len(predicted_sentence))
    
    fontdict = {"fontsize": 14}
    ax.set_xticklabels([' '] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([' '] + predicted_sentence, fontdict=fontdict)
    
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
    
    plt.show()

In [ ]:
def translate(sentence):
    predicted_text, sentence, attention_plot = predict(sentence)
        
    attention_plot = attention_plot[:len(predicted_text.split(' ')), :len(sentence.split(' '))]
    return predicted_text, attention_plot

In [ ]:
random_id = np.random.randint(0, testX.shape[0])
sentence = ' '.join([fr_tokenizer.index_word[i[0]] for i in testX[random_id] if i[0] != 0][1:-1])

In [ ]:
predicted_text, attention_plot = translate(sentence)

print('Input:        ', sentence)
print('Predicted:   ', predicted_text)
print('Ground Truth: ', decode_sequences(eng_tokenizer, testY[random_id, : ,0]))

In [ ]:
plot_attention(attention_plot, sentence, predicted_text)

In [ ]:
data = []

references = []
candidates = []

for i in tqdm(range(testX.shape[0]//2)):
    textX_decoded = decode_sequences(fr_tokenizer, testX[i, : ,0])
    testY_decoded = decode_sequences(eng_tokenizer, testY[i, : ,0])
    candidate = translate(textX_decoded)[0].replace('<end>', '').replace('<start>', '').strip()
    
    data.append({
        'Context': textX_decoded,
        'Reference': testY_decoded,
        'Candidate': candidate,
        'length': len(textX_decoded.split())
    })
    
    references.append([testY_decoded])
    candidates.append(candidate)

In [ ]:
# split into small dataset based on the sentences length
length_ranges = [(1, 5), (6, 10), (11, 15), (16, 20), (21, 30), (31, 40), (41, 60), (61, float('inf'))]

small_datasets = {}
for min_len, max_len in length_ranges:
    filtered_examples = [example for example in data if example['length'] >= min_len and example['length'] <= max_len]
    small_datasets[f'dataset_{min_len}_{max_len}'] = filtered_examples

samples_per_range = []
for key, dataset in small_datasets.items():
    samples_per_range.append(len(dataset))
    print(f"{key}: {len(dataset)} samples")

### Troubleshooting GPU Detection

Your system shows NVIDIA driver 535.230.02 with CUDA 12.2, but TensorFlow 2.19.0 is built against CUDA 12.5 and cuDNN 9, causing a version mismatch. To enable GPU support, choose one of the following:

1. Install CUDA Toolkit 12.5 and cuDNN 9:
   - Download and install CUDA 12.5 from NVIDIA.
   - Download cuDNN 9 for CUDA 12.5 and place headers/libs under `/usr/local/cuda-12.5/`.
   - Add to your `~/.bashrc`:
     ```bash
     export PATH=/usr/local/cuda-12.5/bin:$PATH
     export LD_LIBRARY_PATH=/usr/local/cuda-12.5/lib64:$LD_LIBRARY_PATH
     ```
   - Reload your shell or restart.
   - Reinstall or upgrade TensorFlow:
     ```bash
     pip install --upgrade tensorflow
     ```

2. Or install a TensorFlow build matching CUDA 12.2:
   ```bash
   pip install tensorflow==2.13
   ```

3. Restart the notebook kernel, then rerun the GPU check:
   ```python
   import tensorflow as tf
   print("Detected GPU devices:", tf.config.list_physical_devices('GPU'))
   ```
4. Ensure the CUDA toolkit runtime is installed and accessible:
    - On Ubuntu/Debian, for CUDA 12.2:
      ```bash
      sudo apt-get update
      sudo apt-get install -y cuda-toolkit-12-2
      ```
    - Confirm `/usr/local/cuda` points to CUDA 12.2:
      ```bash
      ls -l /usr/local/cuda
      nvcc --version
      ```
    - If needed, create or update the symlink:
      ```bash
      sudo ln -sf /usr/local/cuda-12.2 /usr/local/cuda
      sudo ldconfig
      ```
    - Reload your shell and rerun the GPU check cell